# Análisis de Ventas con SQL + Pandas

In [41]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import os
import zipfile
import requests

os.makedirs("../data",exist_ok=True)
os.makedirs("../results",exist_ok=True)

In [40]:
# Descargar los datos
url = "https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip"
zip_path = "../data/online_retail_II.zip"

# Descargar el archivo
response = requests.get(url)
with open(zip_path, 'wb') as f:
    f.write(response.content)
print(f"Archivo descargado: {zip_path}")

# Extraer el contenido del .zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("../data")
    extracted_file = os.path.join("../data", "online_retail_II.xlsx")
print(f"Archivo extraído: {extracted_file}")

# Eliminar el .zip original
os.remove(zip_path)
print(f"Archivo .zip eliminado: {zip_path}")

KeyboardInterrupt: 

In [72]:
extracted_file = os.path.join("../data", "online_retail_II.xlsx")
# Cargar el archivo de Excel
archivo_excel = pd.ExcelFile(extracted_file)

# Ver el número de hojas
numero_hojas = len(archivo_excel.sheet_names)
print(f"El archivo tiene {numero_hojas} hojas.")

# Ver los nombres de las hojas
nombres_hojas = archivo_excel.sheet_names
print("Nombres de las hojas:", nombres_hojas)

# Leer la segunda hoja (índice 1) y guardar como CSV
df = pd.read_excel(extracted_file, sheet_name=1)
df
#file_path = "../data/online_retail_II.csv"
#df.to_csv(file_path, index=False)
#print(f"Archivo CSV guardado en: {file_path}")

El archivo tiene 2 hojas.
Nombres de las hojas: ['Year 2009-2010', 'Year 2010-2011']


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [93]:
df.isna().sum()

Invoice           0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
Price             0
Customer_ID       0
Country           0
dtype: int64

In [95]:
display(df)
df.columns

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France


Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer_ID', 'Country'],
      dtype='object')

In [96]:
# pasar a InvoiceDate a fecha y Customer ID a int
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer_ID  541910 non-null  object        
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB


In [97]:
# ¿Cuáles son los ingresos totales?
query = """
    SELECT
       SUM(Quantity * Price) AS total_
    FROM df;
    """
duckdb.sql(query).df()

,total_
0,9.747766e+06


In [98]:
# ¿Qué países generan más ventas?
query = """
    SELECT
        Country,
        SUM(Quantity * Price) AS total_ventas
    FROM df
    GROUP BY Country
    ORDER BY total_ventas DESC;
"""
duckdb.sql(query).df().head(10)

,Country,total_ventas
0,United Kingdom,8.187806e+06
1,Netherlands,2.846615e+05
2,EIRE,2.632768e+05
3,Germany,2.216982e+05
4,France,1.974219e+05
5,Australia,1.370773e+05
6,Switzerland,5.638535e+04
7,Spain,5.477458e+04
8,Belgium,4.091096e+04
9,Sweden,3.659591e+04


In [99]:
# ¿Cuáles son los productos más vendidos?
query="""
    SELECT
        StockCode,
        Description,
        SUM(Quantity) AS TotalQuantity
    FROM
        df
    GROUP BY
        StockCode, Description
    ORDER BY
        TotalQuantity DESC
    LIMIT 10;
    """
duckdb.sql(query).df().head(10)

,StockCode,Description,TotalQuantity
0,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847.0
1,85099B,JUMBO BAG RED RETROSPOT,47363.0
2,84879,ASSORTED COLOUR BIRD ORNAMENT,36381.0
3,22197,POPCORN HOLDER,36334.0
4,21212,PACK OF 72 RETROSPOT CAKE CASES,36039.0
5,85123A,WHITE HANGING HEART T-LIGHT HOLDER,35025.0
6,23084,RABBIT NIGHT LIGHT,30680.0
7,22492,MINI PAINT SET VINTAGE,26437.0
8,22616,PACK OF 12 LONDON TISSUES,26315.0
9,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24753.0


In [100]:
# ¿Cómo evolucionan las ventas en el tiempo?
query="""
    SELECT
        strftime('%Y-%m', InvoiceDate) AS Month,
        SUM(Quantity * Price) AS Total
    FROM df
    GROUP BY Month
    ORDER BY Month;
    """

duckdb.sql(query).df().head(10)

,Month,Total
0,2010-12,748957.020
1,2011-01,560000.260
2,2011-02,498062.650
3,2011-03,683267.080
4,2011-04,493207.121
5,2011-05,723333.510
6,2011-06,691123.120
7,2011-07,681300.111
8,2011-08,682680.510
9,2011-09,1019687.622


In [ ]:
# ¿Qué clientes compran con mayor frecuencia?
query="""
    SELECT
        'Customer ID' ,
        COUNT(DISTINCT Invoice) AS PurchaseFrequency
    FROM
        df
    GROUP BY
        'Customer ID'
    ORDER BY
        PurchaseFrequency DESC
    LIMIT 10;
    """

duckdb.sql(query).df().head(10)

,Customer,PurchaseFrequency
0,Customer ID,25900
